# Исследование надёжности заёмщиков


Целью исследования надежности заемщиков является выявление связей между персональными особенностями заемщика и возвратом кредита срок.
Я постараюсь продемонстрировать зависимость надежности заёмщика от наличия детей, семейного положения, дохода и целей. Рассчитаю долю должников и подкреплю результаты объективными показателями.


# Содержание
1. [Общая информация](#paragraph1)
2. [Предобработка данных](#paragraph2)
    1. [Обработка пропусков](#subparagraph1)
    2. [Замена типа данных](#subparagraph2)
    3. [Обработка дубликатов](#subparagraph3)
    4. [Лемматизация](#subparagraph4)
    5. [Категоризация данных](#subparagraph5)
3. [Ответы на вопросы](#paragraph4)
4. [Общий вывод](#paragraph4)


### Общая информация. <a name="paragraph1"></a>

In [13]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [14]:
data = pd.read_csv('/datasets/data.csv')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [15]:
display(data.info())
display(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод

Файл содержит 21525 строк, 12 колонок. Присутствуют категориальные переменные: образование, пол, семейное положение, занятость, цель кредита и их id. К количественным относятся: количество детей, стаж, возраст, доход. Логическая переменная одна - это наличие задолженности.
В данных есть несколько проблем: отрицательные и нецелые значения стажа, пропущенные значения в стаже и доходе, значения в столбце с образованием записаны разным регистром, цели кредита не категоризированы, что затрудняет анализ. 

### Предобработка данных <a name="paragraph2"></a>

### Обработка пропусков <a name="subparagraph1"></a>

Обработаем пропуски в данных. Проведем их подсчет, взглянем на несколько строк с пустыми значениями и придумаем стратегию их заполнения.

In [16]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [17]:
data[data['days_employed'].isnull()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Принято решение заполнить пропуски в столбце **'days_employed'** средним значениями в зависимости от **'income_type'** (профессии или типа занятости).

In [18]:
days_employed_agg = data.groupby('income_type')['days_employed'].mean()
days_employed_agg

income_type
безработный        366413.652744
в декрете           -3296.759962
госслужащий         -3399.896902
компаньон           -2111.524398
пенсионер          365003.491245
предприниматель      -520.848083
сотрудник           -2326.499216
студент              -578.751554
Name: days_employed, dtype: float64

In [19]:
professions = data['income_type'].unique() #переменная со списком профессий
for element in professions: 
    data.loc[(data['income_type'] == element)&(data['days_employed'].isnull()), 'days_employed'] = days_employed_agg[element]
data.isna().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Для наглядности создадим столбец **'years_employed'** со стажем по годам, разделив на 365 дней.

In [20]:
data['years_employed'] = data['days_employed'] // 365 
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,-24.0
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,-12.0
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,-16.0
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,-12.0
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,932.0


Далее обработаем пропуски в столбце **'total_income'** с доходами. роанализируем и разработаем стратегию.
Было принято решение заполнить эти пропуски медианными значениями дохода в зависимости от профессии.

In [21]:
data[data['total_income'].isnull()].head() 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
12,0,365003.491245,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу,1000.0
26,0,-3399.896902,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование,-10.0
29,0,365003.491245,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости,1000.0
41,0,-3399.896902,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем,-10.0
55,0,365003.491245,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу,1000.0


Выделим переменную, которая будет содержать в себе профессии и медианный уровень дохода по ним.

In [22]:
total_income_agg = data.groupby('income_type')['total_income'].median() 
total_income_agg 

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [23]:
for element in professions:
    data.loc[(data['income_type'] == element)&(data['total_income'].isnull()), 'total_income'] = total_income_agg[element]
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
years_employed      0
dtype: int64

### Вывод

Обнаружено равное количество пропущенных значений в столбцах Стаж и Доход. Вывод строк с пропущенными значениями показал, что Стаж и доход пропущены в одних и тех же местах. В тоже время другие категории показывают, что это абсолютно разные люди. Полагаю, что пропущенные значения возникли, потому что в этих строках - новые клиенты банка, которые еще не передали информацию о доходе и стаже, либо их персональные данные обновились.
Исключать из подборки этих людей нельзя, т.к. они составляют большую часть выборки. 
Заменим пропущенные количественные значения характерными данными. Возьмем средний стаж по типу занятости и медианный доход по типу занятости.
Хотела применить метод fillna(), но он меняет пустые значения всего столбца, а мои значения зависят от типа занятости. Типов занятости аж 8, поэтому писать код для замены по каждому было бы громоздко. Поэтмоу был создан цикл, который заменяет пустые значения на нужные сразу.

Пустые значения успешно заполнены данными.

### Замена типа данных <a name="subparagraph2"></a>

В прошлых выводах датафрейма заметно насколько громоздкими выглядят данные по столбцам со стажем **'days_employed'** и с доходами **'total_income'**. Изменим типы данных на целочисленный int64, чтобы анализировать цифры было удобнее.

In [25]:
data.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
years_employed      float64
dtype: object

Изменим типы данных в столбцах **'days_employed', 'total_income' и 'years_employed'** с помощью метода astype().

In [26]:
data['days_employed'] = data['days_employed'].astype(int)

In [27]:
data['total_income'] = data['total_income'].astype(int)

In [28]:
data['years_employed'] = data['years_employed'].astype(int)

In [29]:
data.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
years_employed       int64
dtype: object

### Вывод

Замена типов данных сделала датафрейм читабельным. 
Просмотр данных теперь будет ускорен.

### Обработка дубликатов <a name="subparagraph3"></a>

Обработаем дубликаты в данных. 

Подсчет полных дубликатов

In [30]:
data.duplicated().sum()

54

Удаление полных дубликатов

In [31]:
data.drop_duplicates().reset_index(drop = True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,-24
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,-12
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,-16
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,-12
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,-13
21467,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,942
21468,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,-6
21469,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,-9


Проверим регистр строковых значений столбца **'education'**.

In [32]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Обнаружено много одинаковых значений разным регистром, из-за чего возникли "ложноуникальные" категории образования.
Унифицируем и приведем все к нижнему регистру.
Снова проведем проверку на полные дубликаты и избавимся от них.

In [33]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [34]:
data.duplicated().sum()

71

In [35]:
data.drop_duplicates().reset_index(drop = True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,-24
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,-12
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,-16
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,-12
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,-13
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,942
21451,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,-6
21452,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,-9


### Вывод

Метод duplicated() по всему Датафрейму обнаружил 54 полных дубликата. Можем смело их удалить методом drop_duplicates(), потому что вероятность совпадения стольких разных данных по разным людям очень мала, значит, вероятнее всего, это действительно дубликаты. Причина их появления может заключаться как в системной ошибке, так и в том, что клиент подавал заявку на кредит несколько раз в один и тот же банк, а банк несколько раз его зарегистрировал. reset_index() используем для перезаписи индексов.
Ранее наблюдали, что в столбце с образованием содержались данные с разным регистром букв, поэтому унифицировали, приведя к общему регистру. Удалили еще 71 полный дубликат. 
Теперь данные очищены и уникальный.

### Лемматизация <a name="subparagraph4"></a>

Столбец **'purpose'**, содержащий в себе цели кредита, содержит похожие данные, но для кода они уникальны. Ставим задачу провести лемматизацию целей, чтобы у нас появилась возможность категоризировать и подсчитать количество займов по целям.
Проводим лемматизацию с помощью библиотеки **pymystem3**. Для этого напишем функцию, применим к столбцу с целями и запишем наш результат в новый столбец.

In [36]:
m = Mystem()
def lemmatized(element):
    result = m.lemmatize(element)
    return result
data['purpose_lemmas'] = data['purpose'].apply(lemmatized)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,purpose_lemmas
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,-24,"[покупка, , жилье, \n]"
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,-12,"[приобретение, , автомобиль, \n]"
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,-16,"[покупка, , жилье, \n]"
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,-12,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932,"[сыграть, , свадьба, \n]"


In [37]:
list_of_purpose_lemmas = list(data['purpose_lemmas'])
# list_of_purpose_lemmas
final_list_of_purpose_lemmas = []
for item in list_of_purpose_lemmas: 
    final_list_of_purpose_lemmas += item 
# final_list_of_purpose_lemmas
print(Counter(final_list_of_purpose_lemmas)) # посчитываем число упоминаний лемм


Counter({' ': 33677, '\n': 21525, 'недвижимость': 6367, 'покупка': 5912, 'жилье': 4473, 'автомобиль': 4315, 'образование': 4022, 'с': 2924, 'операция': 2610, 'свадьба': 2348, 'свой': 2235, 'на': 2233, 'строительство': 1881, 'высокий': 1375, 'получение': 1316, 'коммерческий': 1315, 'для': 1294, 'жилой': 1233, 'сделка': 944, 'дополнительный': 909, 'заниматься': 908, 'проведение': 777, 'сыграть': 774, 'сдача': 653, 'семья': 641, 'собственный': 635, 'со': 630, 'ремонт': 612, 'подержанный': 489, 'подержать': 479, 'приобретение': 462, 'профильный': 436})


### Вывод

В разрозненных целях кредита провели лемматизицию, тем самым унифицировав запись данных, сделали данные читабельными для кода.  Для подсчета уникальных лемм объединили все леммы в единый список с помощью цикла. Подсчитали с помощью Counter и наглядно выявили популярность целей.

**Заемщики чаще всего берут кредиты на:**
* покупку недвижимости
* автомобиль
* образование
* свадьбу
* ремонт

### Категоризация данных <a name="subparagraph5"></a>

Проведем категоризацию данных по нескольким признакам исходя из целей исследования. Это поможет нам сгруппировать данные, систематизировать их и быстро сделать выводы.

Выделим категории по:
* наличию детей
* целям
* уровням дохода

#### Категоризация по признаку "наличие детей".

In [38]:
def children_group(children): 
    if children == 0:
        return 'бездетные'
    if children < 0:
        return 'ошибка'
    return 'с детьми'
# print(children_group(5))
data['children_grouped'] = data['children'].apply(children_group) # записываем данные в новый столбец
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,purpose_lemmas,children_grouped
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,-24,"[покупка, , жилье, \n]",с детьми
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,-12,"[приобретение, , автомобиль, \n]",с детьми
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,-16,"[покупка, , жилье, \n]",бездетные
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,-12,"[дополнительный, , образование, \n]",с детьми
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932,"[сыграть, , свадьба, \n]",бездетные


In [64]:
data.groupby('children_grouped')['debt'].count() #Проверка 

children_grouped
бездетные    14149
ошибка          47
с детьми      7329
Name: debt, dtype: int64

In [39]:
def fix(children): # пофиксим ошибку с детьми - переведем отрицательных в положительные
    if children < 0:
        children = children * (-1)
    return children
data['children'] = data['children'].apply(fix)

In [40]:
data['children_grouped'] = data['children'].apply(children_group)
data.groupby('children_grouped')['debt'].count()

children_grouped
бездетные    14149
с детьми      7376
Name: debt, dtype: int64

#### Категоризация по целям.

In [41]:
def purpose_group(purpose): 
    for item in purpose:
        if 'ремонт' in purpose:
            return 'ремонт'
        if 'недвижимость' in purpose or 'жилье' in purpose:
            return 'недвижимость'
        if 'автомобиль' in purpose:
            return 'автомобиль'
        if 'образование' in purpose:
            return 'образование'
        if 'свадьба' in purpose:
            return 'свадьба'
    return 'неопеределено'

data['purpose_grouped'] = data['purpose_lemmas'].apply(purpose_group)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,purpose_lemmas,children_grouped,purpose_grouped
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,-24,"[покупка, , жилье, \n]",с детьми,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,-12,"[приобретение, , автомобиль, \n]",с детьми,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,-16,"[покупка, , жилье, \n]",бездетные,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,-12,"[дополнительный, , образование, \n]",с детьми,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932,"[сыграть, , свадьба, \n]",бездетные,свадьба


In [42]:
data.groupby('purpose_grouped')['total_income'].count()

purpose_grouped
автомобиль       4315
недвижимость    10228
образование      4022
ремонт            612
свадьба          2348
Name: total_income, dtype: int64

#### Категоризация по уровню доходов

Принято решение разделить доходы на 4 разные группы и дать им условные обозначения.

In [43]:
total_income_max = data['total_income'].max() 
total_income_max

2265604

In [44]:
total_income_min = data['total_income'].min()
total_income_min

20667

In [45]:
len(data['total_income'])//4 

5381

In [46]:
filtred_data = data[['total_income', 'purpose']] 
filtred_data_sorted = filtred_data.sort_values(by = 'total_income').reset_index(drop=True) 
filtred_data_sorted.head()

,total_income,purpose
0,20667,недвижимость
1,21205,заняться высшим образованием
2,21367,приобретение автомобиля
3,21695,на проведение свадьбы
4,21895,недвижимость


Определение границ групп.

In [47]:
first_group_board = filtred_data_sorted.loc[5381, 'total_income'] # 107798 граница первой группы
second_group_board = filtred_data_sorted.loc[10764, 'total_income'] # 142594 граница второй группы
third_group_board = filtred_data_sorted.loc[16145, 'total_income'] # 195607 граница третей группы

In [48]:
def income_group(total_income):
    if total_income <= first_group_board:
        return 'бедные'
    if first_group_board < total_income <= second_group_board:
        return 'средний класс'
    if second_group_board < total_income <= third_group_board:
        return 'богатые'
    return 'элита'
data['income_grouped'] = data['total_income'].apply(income_group)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,purpose_lemmas,children_grouped,purpose_grouped,income_grouped
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,-24,"[покупка, , жилье, \n]",с детьми,недвижимость,элита
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,-12,"[приобретение, , автомобиль, \n]",с детьми,автомобиль,средний класс
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,-16,"[покупка, , жилье, \n]",бездетные,недвижимость,богатые
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,-12,"[дополнительный, , образование, \n]",с детьми,образование,элита
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932,"[сыграть, , свадьба, \n]",бездетные,свадьба,богатые


### Вывод

Для ответа на последующие вопросы была проведена категоризация по 3м признакам: наличие детей, цели кредита, уровень дохода. 
По наличию детей заемщиков поделили на бездетных и с детьми. Обнаружили ошибку с отрицательными детьми и поправили ее.

Выделили 5 целей получения кредита и присвоили их: недвижимость, авто, свадьба, образование, ремонт.
По доходам было решено поделить на 4 равные группы. Названия групп условные: бедные, средний класс, богатые, элита. 

### Ответы на вопросы <a name="paragraph3"></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [49]:
children_and_debt = data.groupby('children_grouped').agg({'debt': ['count', 'sum']})
children_and_debt

debt      
                  count   sum
children_grouped             
бездетные         14149  1063
с детьми           7376   678

In [50]:
children_and_debt['part'] = children_and_debt['debt']['sum'] / children_and_debt['debt']['count']
children_and_debt['part'] = children_and_debt['part'] *100
children_and_debt.sort_values(by = 'part')

debt            part
                  count   sum          
children_grouped                       
бездетные         14149  1063  7.512898
с детьми           7376   678  9.191974

### Вывод

По имеющейся у нас выборке можно сделать вывод, что среди заемщиков с детьми дожников больше, чем среди заемщиков, у которых детей нет.
Предположительно это связано с тем, что наличие детей приводит к спонтанным тратам, и это отнимает средства, которые должны были быть пущены на возврат долгов

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [51]:
family_and_debt = data.groupby('family_status').agg({'debt': ['count', 'sum']})
family_and_debt

debt     
                       count  sum
family_status                    
Не женат / не замужем   2813  274
в разводе               1195   85
вдовец / вдова           960   63
гражданский брак        4177  388
женат / замужем        12380  931

In [52]:
family_and_debt['part'] = family_and_debt['debt']['sum'] / family_and_debt['debt']['count']
family_and_debt['part'] = family_and_debt['part'] *100
family_and_debt.sort_values(by = 'part')

debt           part
                       count  sum          
family_status                              
вдовец / вдова           960   63  6.562500
в разводе               1195   85  7.112971
женат / замужем        12380  931  7.520194
гражданский брак        4177  388  9.288963
Не женат / не замужем   2813  274  9.740491

### Вывод

В нашей выборке довольно ответственные заемщики, резких перекосов в группах по семейному положению не наблюдается. Доля задолжавших держится на уровне до 10% от общего числа взявших кредит. 
Но все же люди в браке или бывшие в браке возвращают кредиты лучше, чем те, кто никогда не был официально женат или замужем. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [53]:
income_and_debt = data.groupby('income_grouped').agg({'debt': ['count', 'sum']})
income_and_debt

debt     
               count  sum
income_grouped           
бедные          5382  427
богатые         5241  446
средний класс   5523  483
элита           5379  385

In [54]:
income_and_debt['part'] = income_and_debt['debt']['sum'] / income_and_debt['debt']['count']
income_and_debt['part'] = income_and_debt['part'] *100
income_and_debt.sort_values(by = 'part')

debt           part
               count  sum          
income_grouped                     
элита           5379  385  7.157464
бедные          5382  427  7.933854
богатые         5241  446  8.509826
средний класс   5523  483  8.745247

### Вывод

Судя по нашим результатам, самые надежные заемщики - это элита, самые обеспеченные люди, процент долгов у них меньше всего. Значит зависимость здесь прямая - чем больше доход, тем лучше возвращаются кредиты. 

- Как разные цели кредита влияют на его возврат в срок?

In [55]:
purpose_and_debt = data.groupby('purpose_grouped').agg({'debt': ['count', 'sum']})
purpose_and_debt

debt     
                 count  sum
purpose_grouped            
автомобиль        4315  403
недвижимость     10228  747
образование       4022  370
ремонт             612   35
свадьба           2348  186

In [56]:
purpose_and_debt['part'] = purpose_and_debt['debt']['sum'] / purpose_and_debt['debt']['count']
purpose_and_debt['part'] = purpose_and_debt['part'] *100
purpose_and_debt.sort_values(by = 'part')

debt           part
                 count  sum          
purpose_grouped                      
ремонт             612   35  5.718954
недвижимость     10228  747  7.303481
свадьба           2348  186  7.921635
образование       4022  370  9.199403
автомобиль        4315  403  9.339513

### Вывод

Возврат кредита также зависит и от целей его получения. Наши результаты говорят о том, что охотнее возвращают деньги взятые на ремонт, чуть менее охотно, если взяли на покупку недвижимости или проведение свадьбы. Самые злостные должники - те, кто решил обновить автомобиль или оплатить образование. 


### Общий вывод <a name="paragraph4"></a>

Проведенный анализ влияния различных факторов на возрат кредитов в срок позволил выявить важнейшие показатели, которые должны быть учтены при выдаче кредита и формирования кредитного рейтинга.

#### Семейные факторы.
##### Наличие детей.

| Наличие детей | Процент должников |
|:-------------:|:---------------:|
| с детьми       | 9,2% |
| без детей     | 7,5%        |

Среди заёмщиков с детьми задерживают возврат **9,2%** людей, среди же бездетных этот показатель меньше на **1,7%** и составляет **7,5%** от общего числа бездетных.
Таким образом, наличие детей - признак отрицательный.

##### Семейное положение.
Рейтинг должников сформировался так:

| Семейное положение | Процент должников |
|:-------------:|:---------------:|
| вдовцы        | 6,5% |
| в разводе     | 7,1%        |
| женатые       | 7,5%        |
| гражданский брак     | 9,3%        |
| неженатые      | 9,7%        |

    
Получается, что те, кто в браке или состоял в нем ранее - более надежные заемщики. И это положительный признак для выдачи займа.

#### Доходы.

| Группа по доходу | Процент должников |
|:-------------:|:---------------:|
| элита        | 7,2% |
| бедные    | 7,9%        |
| богатые       | 8,5%        |
| средний класс     | 8,7%        |

Поделив всю выборку на 4 равные группы, мы получили интересные результаты по доходам.
Очевидно было предположить, что самые богатые возвращают деньги лучше других, процент должников составляет всего 7,2%. Эта гипотеза верна.
Но интереснее то, что на втором месте рейтинга по надежности идут условно бедные, где должников 7,9%. Оказалось те, кто не располагает большим количеством денежных средств, лучше ими распоряжается благодаря трудным жизненным условиям. 

Третье и четвертое места заняли средние категории с близкими показателями.


#### Цели.


Исследование возвратов кредитов по целям оказался не менее интересным, оно продемонстрировало, что субъективные оценки целей не должны быть опеределяющими и решающими. Цифры показывают иное.

| Цель | Процент должников |
|:-------------:|:---------------:|
| ремонт        | 5,7% |
| недвижимость    | 7,3%        |
| свадьба       | 7,9%        |
| автомобиль     | 9,1%        |
| образование      | 9,3%        |

Так, цель **свадьба**, которую бы многие не одобрили, находится ровно посередине рейтинга. Процент должников составляет 7,9%
Самые надежные цели - это **ремонт** 5,7% и **недвижимость** 7,3%. Такие цели - это всегда вклад или инвестиция на будущее, к ней люди подходят ответственно. К тому же чаще всего они не требуют дополнительных вложений. В отличие от **образования** (9,3% долгов) и **автомобиля** (9,1% долгов). С покупкой автомобиля траты не заканчиваются, а наоборот возрастают, это средство передвижения, которое требует постоянных вливаний средств, к тому же автомобиль с каждым годом дешевеет. Образование в свою очередь очень полезно, но брать кредит на него рискованно из-за отсутсвия гарантий трудоустройста и низкой оплаты труда в первые годы работы.

Исходя из полученных результатов, можно сделать вывод о том, что на возврат кредита в срок влиют все перечисленные факторы. При принятии решения о выдаче заемных средств нужно учитывать всю информацию о заемщике: семейное положение, доходы, наличие детей, цели кредита. Данные о человеке + результаты нашего исследования помогут сформировать кредитный портрет заемщика и дать ответы на вопросы: потенциально надежный человек перед нами или нет. Какую сумму ему можно одобрить, под какой залог и на какой срок.